In [3]:
from sqlalchemy import create_engine, MetaData, Table, Column
from sqlalchemy import DateTime, Float
import pandas as pd

# --------------------------------------------------------
# 1. CONNECTION TO AIVEN MYSQL
# --------------------------------------------------------
host = "anly-615-project-anlyproject.g.aivencloud.com"
port = 23263
user = "avnadmin"
password = "AVNS_uZtAlXsQZVgdnkwXesP"
database = "defaultdb"

# SSL argument is required for Aiven
connect_args = {"ssl": {"ssl_mode": "REQUIRED"}}

engine = create_engine(
    f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}",
    connect_args=connect_args,
    echo=False
)

metadata = MetaData()

# --------------------------------------------------------
# 2. DEFINE TABLE SCHEMA
# --------------------------------------------------------
# We define the schema for the cleaned SP500 data
sp500_table = Table(
    "sp500_cleaned",
    metadata,
    Column("date", DateTime, primary_key=True),
    Column("close", Float)
)

# --------------------------------------------------------
# 3. CREATE TABLE IN MYSQL (only if not exists)
# --------------------------------------------------------
metadata.create_all(engine)
print("Table 'sp500_cleaned' created (if not already present).")

# --------------------------------------------------------
# 4. LOAD AND CLEAN DATA (Your Code Logic)
# --------------------------------------------------------

# Load the CSV file (Assuming SP500.csv is in the same directory)
# You can update the path below if the file is elsewhere, e.g., r"C:\Users\...\SP500.csv"
csv_path = r"C:\Users\nurta\Downloads\SP500.csv"
df = pd.read_csv(csv_path)

# --- Cleaning Steps from your notebook ---

# 1. Remove columns open, high, low, volume
columns_to_remove = ["open", "high", "low", "volume"]
df = df.drop(columns=columns_to_remove, errors='ignore')

# 2. Drop rows with any null values
df = df.dropna()

# 3. Ensure correct data types
df['close'] = df['close'].astype(float)

# 4. Convert date to datetime objects
# Note: For SQL upload via SQLAlchemy, it is best to keep these as Python datetime objects
# rather than converting them to strings (YYYY-MM-DD).
df['date'] = pd.to_datetime(df['date'], utc=True)

# Remove timezone info if your MySQL server expects naive datetimes, 
# or keep it if the column supports it. Aiven usually handles standard datetime.
df['date'] = df['date'].dt.tz_convert(None) 

# Remove duplicates to prevent Primary Key errors
df = df.drop_duplicates(subset="date")

print(f"Data cleaned. Rows to upload: {len(df)}")

# --------------------------------------------------------
# 5. INSERT INTO MYSQL
# --------------------------------------------------------

df.to_sql(
    "sp500_cleaned",
    con=engine,
    if_exists="append",  # Use 'replace' if you want to overwrite the table each time
    index=False
)

print("SP500 data uploaded successfully!")

# --------------------------------------------------------
# 6. VALIDATION QUERIES
# --------------------------------------------------------

print("\n--- VALIDATION: Check uploaded table ---")
# Check the first 5 rows
sample_sp500 = pd.read_sql("SELECT * FROM sp500_cleaned ORDER BY date ASC LIMIT 5;", engine)
print(sample_sp500)

print("\nAll Done!")

Table 'sp500_cleaned' created (if not already present).
Data cleaned. Rows to upload: 24589
SP500 data uploaded successfully!

--- VALIDATION: Check uploaded table ---
                 date  close
0 1927-12-30 05:00:00  17.66
1 1928-01-03 05:00:00  17.76
2 1928-01-04 05:00:00  17.72
3 1928-01-05 05:00:00  17.55
4 1928-01-06 05:00:00  17.66

All Done!
